# Перевірка статистичної гіпотези про вигляд розподілу

Спостерігається вибірка $X_1,\dots,X_n$, де $\{X_i\}$ – незалежні однаково розподілені випадкові величини, які мають показниковий розподіл з параметром $\lambda$, тобто
 $F(u,\lambda)=\mathbb{P}(X_i\leq u)=1-e^{-u\lambda}, u\geq 0$

Перевірку статистичних гіпотез вести при рівні значимості $\gamma=0.05$. Кожне з
наступних чотирьох завдань виконувати для $n=1000$, $n=10\ 000$ та $n=100\ 000$.

Користуючись перетворенням $Y_i=F(X_i,\lambda), i=1,\dots,n$ перевіряти на рівномірність
випадкові величини ${Y_i}$ (<u>лише перші три завдання</u>).

## Завдання 1
За допомогою критерія Колмогорова перевірити гіпотези:

a) $H_0: X_i\sim F(u;1)$, коли насправді $X_i\sim F(u;1)$

b) $H_0: X_i\sim F(u;1)$, коли насправді $X_i\sim F(u;1.2)$

## Алгоритм перевірки гіпотези $H_0$ за Критерієм Колмогорова
1. **Обираємо рівень значимості** $\gamma$ та за таблицею для розподілу Колмогорова знаходимо $z_\gamma$ таке, що:
$\quad
K(z_\gamma) = 1 - \gamma
$
2. **Обчислюємо** обчислюємо $$D_n(x)=\max\limits_{1\leq k\leq n} \left\{\max\left\{\left(F(x_{(k)})-\frac{k-1}{n}\right),\left(\frac{k}{n}-F(x_{(k)})\right)\right\}\right\}$$
3. Якщо 
$
\sqrt{n} D_n(x) < z_\gamma
$
то робимо висновок: статистичні дані **не суперечать** гіпотезі $H_0$.  
Якщо ж $x \in \mathbb{Z}_1$, то слід прийняти альтернативну гіпотезу $H_1$.


Критична область $\mathbb{Z}_1=\{x: \sqrt{n}D_n(x)\geq z_\gamma\}$

In [1]:
import numba
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

@numba.njit
def ks_statistic(sample, λ):
    sorted_sample = np.sort(sample)
    n = len(sample)

    F = 1 - np.exp(-λ * sorted_sample)
    d = 0.0
    for i in range(n):
        d = max(
            d,
            F[i] - i / n,
            (i + 1) / n - F[i]
        )
    return d

def ks_statistic_lib(sample, λ):
    F = lambda x: 1 - np.exp(-λ*x)
    d, _ = stats.kstest(sample, F)
    return d

In [12]:
lambda_lst = [1, 1.2]
n_trials_lst = [1000, 10000, 100000]

gamma = 0.05
z_gamma = 1.36  # для gamma=0.05

print(f"{'Розмір вибірки':^20} {'λ':^10} {'√n * D_n':^15} {'Порівняння':^15} {'Критичне значення':^15} {'Висновок':^30}")
print("=" * 120)

for n in n_trials_lst:
    for λ in lambda_lst:
        sample = np.random.exponential(1/λ, n)
        D_n = ks_statistic(sample, 1.0)

        value = np.sqrt(n) * D_n
        condition = bool(value >= z_gamma)
        comparison = "≥" if condition else "<"
        conclusion = "Приймаємо альтернативну гіпотезу" if condition else "Приймаємо нульову гіпотезу"

        print(f"{n:^20} {λ:^10.2f} {value:^15.5f} {comparison:^15} {z_gamma:^20.5f} {conclusion:^30}")
    
    print("-" * 120)


   Розмір вибірки        λ         √n * D_n       Порівняння    Критичне значення            Висновок           
        1000            1.00        0.88652            <              1.36000          Приймаємо нульову гіпотезу  
        1000            1.20        1.94448            ≥              1.36000        Приймаємо альтернативну гіпотезу
------------------------------------------------------------------------------------------------------------------------
       10000            1.00        1.23925            <              1.36000          Приймаємо нульову гіпотезу  
       10000            1.20        7.45595            ≥              1.36000        Приймаємо альтернативну гіпотезу
------------------------------------------------------------------------------------------------------------------------
       100000           1.00        1.09098            <              1.36000          Приймаємо нульову гіпотезу  
       100000           1.20       21.42180            ≥     

## Завдання 2
За допомогою критерія $\chi^2$ перевірити гіпотези:

a) $H_0: X_i\sim F(u;1)$, коли насправді $X_i\sim F(u;1)$

b) $H_0: X_i\sim F(u;1)$, коли насправді $X_i\sim F(u;1.2)$

<u>*Зауваження*</u>: $r=30\cdot\frac{n}{1000}$ $(r=\lfloor 1+\log_2{n}\rfloor)$

## Алгоритм перевірки гіпотези $H_0$ за критерієм $\chi^2$

1. **Обираємо** $\gamma$ та за таблицею для $\chi^2$-розподілу знаходимо $z_\gamma$ таке, що:
$\\
T_{r-1}(z_\gamma) = 1 - \gamma
$

2. **Обчислюємо ймовірності** $\{ p_i \}$ та частоти $\{ \nu_i \}$.

3. **Обчислюємо**  $\Delta_n^{(r)}=\sum_{i=1}^r \frac{(\nu_i-np_i)^2}{np_i}$.

4. Якщо $\Delta_n^{(r)} < z_\gamma$, то робимо висновок: статистичні дані **не суперечать** гіпотезі $H_0$.

5. Якщо ж $\Delta_n^{(r)} \geq z_\gamma$, тобто $\nu \in \mathbb{Z}_1$, то слід прийняти альтернативну гіпотезу $H_1$.

In [3]:
import numpy as np
from scipy import stats

lambda_lst = [1, 1.2]
n_trials_lst = [1000, 10000, 100000]
gamma = 0.05


# Заголовок таблиці
print(f"{'Розмір вибірки':^20} {'λ':^10} {'Статистика χ²':^20} {'Критичне значення':^20} {'Висновок':^30}")
print("=" * 110)

for n in n_trials_lst:
    r = int(1+np.log2(n)) #int(30 * n / 1000)
    z_gamma = stats.chi2.ppf(1 - gamma, df=r-1)

    for λ in lambda_lst:
        # Генеруємо вибірку
        sample = np.random.exponential(scale=1/λ, size=n)

        # Визначаємо межі інтервалів (рівномірно за квантилями)
        bins = np.quantile(sample, np.linspace(0, 1, r + 1))

        # Обчислюємо частоти та очікувані значення
        observed, _ = np.histogram(sample, bins=bins)
        expected = n * np.diff(1 - np.exp(-bins))

        # Обчислюємо значення критерію χ²
        chi2_stat = np.sum((observed - expected) ** 2 / expected)

        # Висновок
        conclusion = "Приймаємо альтернативну гіпотезу" if chi2_stat > z_gamma else "Приймаємо нульову гіпотезу"

        # Вивід у форматі таблиці
        print(f"{n:^20} {λ:^10.2f} {chi2_stat:^20.5f} {z_gamma:^20.5f} {conclusion:^30}")

    print("-" * 110)

   Розмір вибірки        λ         Статистика χ²      Критичне значення              Висновок           
        1000            1.00          7.95976              16.91898         Приймаємо нульову гіпотезу  
        1000            1.20          22.42320             16.91898       Приймаємо альтернативну гіпотезу
--------------------------------------------------------------------------------------------------------------
       10000            1.00          9.68524              22.36203         Приймаємо нульову гіпотезу  
       10000            1.20         281.59217             22.36203       Приймаємо альтернативну гіпотезу
--------------------------------------------------------------------------------------------------------------
       100000           1.00          14.11841             26.29623         Приймаємо нульову гіпотезу  
       100000           1.20         2751.92360            26.29623       Приймаємо альтернативну гіпотезу
-------------------------------------

## Завдання 3
За допомогою критерія пустих ящиків (асимптотична теорема) перевірити гіпотези:

a) $H_0: X_i\sim F(u;1)$, коли насправді $X_i\sim F(u;1)$

b) $H_0: X_i\sim F(u;1)$, коли насправді $X_i\sim F(u;1.2)$

<u>*Зауваження*</u>: $r=\frac{n}{2}, \ \rho=2$

Критична область $\mathbb{Z}_1=\left\{k: k>re^{-\rho}+z_\gamma\sqrt{re^{-\rho}[1-(1+\rho)e^{-\rho}]}\right\}$, де $k$ - кількість пустих ящиків

In [18]:
import numpy as np
from scipy.stats import norm

# Параметри задачі
n_trials_lst = [1000, 10000, 100000]  # Розмір вибірок
lambda_lst = [1.0, 1.2]  # Параметри λ
rho = 2
gamma = 0.05

# Заголовок таблиці
print(f"{'Розмір вибірки':^20} {'λ':^10} {'Кількість пустих ящиків':^30} {'Критичне значення':^25} {'Висновок':^30}")
print("=" * 120)

for n in n_trials_lst:
    r = n // 2
    z_gamma = norm.ppf(1 - gamma)
    critical_value = r * np.exp(-rho) + z_gamma * np.sqrt(r * np.exp(-rho) * (1 - (1 + rho) * np.exp(-rho)))

    for λ in lambda_lst:
        # Генеруємо вибірку з показникового розподілу
        samples = np.random.exponential(scale=1/λ, size=n)
        xi = 1 - np.exp(-1.0*samples)  # F(x) для показникового розподілу

        # Рівномірний поділ у просторі ймовірностей
        bins = np.linspace(0, 1, r + 1)

        # Підрахунок кількості пустих ящиків
        num_observ, _ = np.histogram(xi, bins=bins)
        empty_boxes = np.where(num_observ == 0, 1, 0)
        stat = empty_boxes.sum()

        # Висновок
        conclusion = "Приймаємо нульову гіпотезу" if stat < critical_value else "Приймаємо альтернативну гіпотезу"

        # Вивід у форматі таблиці
        print(f"{n:^20} {λ:^10.2f} {stat:^30} {critical_value:^25.5f} {conclusion:^30}")

    print("-" * 120)

   Розмір вибірки        λ         Кількість пустих ящиків         Критичне значення                Висновок           
        1000            1.00                  71                       78.09583            Приймаємо нульову гіпотезу  
        1000            1.20                  71                       78.09583            Приймаємо нульову гіпотезу  
------------------------------------------------------------------------------------------------------------------------
       10000            1.00                 662                       709.65324           Приймаємо нульову гіпотезу  
       10000            1.20                 737                       709.65324         Приймаємо альтернативну гіпотезу
------------------------------------------------------------------------------------------------------------------------
       100000           1.00                 6791                     6871.04604           Приймаємо нульову гіпотезу  
       100000           1.20        

## Завдання 4
За допомогою критерія однорідності Смирнова перевірити гіпотези

a) $H_0: (X_1^{(1)},\dots,X_n^{(1)})\sim F(u;1) \quad (X_1^{(2)},\dots,X_m^{(2)})\sim F(u;1)$ (саме так ці вибірки і генерувались);

b) $H_0: (X_1^{(1)},\dots,X_n^{(1)})\sim F(u;1) \quad (X_1^{(2)},\dots,X_m^{(2)})\sim F(u;1)$ (насправді $\overline{X}^{(1)}\sim F(u;1) \quad \overline{X}^{(2)}\sim F(u;1.2)$);

<u>*Зауваження*</u>: обрати $m=\frac{n}{2}$

## Алгоритм перевірки гіпотези $H_0$

1. **Обираємо рівень значимості** $\gamma$ та за таблицею для розподілу Колмогорова знаходимо $z_\gamma$ таке, що:

$
K(z_\gamma) = 1 - \gamma.
$

2. **Обчислюємо** $D_{n,m}(\bar{x}, \bar{y})$:

$
D_{n,m}(\bar{x}, \bar{y}) = \max \left( D_{n,m}^+, D_{n,m}^- \right),
$

де  

$
D_{n,m}^+ = \max_{1 \leq k \leq m} \left( \frac{k}{m} - \hat{F}_n^{(1)}(y_{(k)}) \right),
$

$
D_{n,m}^- = \max_{1 \leq k \leq m} \left( \hat{F}_n^{(1)}(y_{(k)}) - \frac{k - 1}{m} \right).
$

$\bar{x}, \bar{y}$ — це реалізації вибірок $\bar{X}, \bar{Y}$, а $\{ y_{(k)} \}$ — порядкові статистики, які відповідають вибірці $\bar{Y}$.

3. **Визначаємо критичну область**:

$
Z_1 = \left\{ (\bar{x}, \bar{y}) : D_{n,m}(\bar{x}, \bar{y}) > z_\gamma \sqrt{\frac{1}{n} + \frac{1}{m}} \right\}.
$

4. Якщо $(\bar{x}, \bar{y}) \notin Z_1$, то робимо висновок: статистичні дані **не суперечать** гіпотезі $H_0$.  
   Якщо ж $(\bar{x}, \bar{y}) \in Z_1$, то слід прийняти **альтернативну гіпотезу** $H_1$.


$$
\hat{F_n}(y)=\frac{\sum_{i=1}^n I\{X_i<y\}}{n}
$$

In [5]:
from scipy.stats import norm
import numpy as np

# Параметри задачі
n_trials_lst = [1000, 10000, 100000]  # Розмір вибірок
lambda_lst = [1.0, 1.2]  # Параметри λ
gamma = 0.05
z_gamma = 1.36 # для gamma=0.05

# Заголовок таблиці
print(f"{'Розмір вибірки (n)':^20} {'λ':^10} {'D':^15} {'Критичне значення':^20} {'Висновок':^30}")
print("=" * 95)

for n in n_trials_lst:
    m = n // 2

    for λ in lambda_lst:
        # Генеруємо вибірки для випадку (b)
        sample_1 = np.random.exponential(scale=1, size=n)     # λ = 1
        sample_2 = np.random.exponential(scale=1/λ, size=m)   # λ = 1.2

        # Сортуємо вибірки
        sample_1 = np.sort(sample_1)
        sample_2 = np.sort(sample_2)

        # Обчислення емпіричної функції розподілу
        F_hat = np.searchsorted(sample_1, sample_2, side='right')/n # швидше ніж np.array([np.sum(sample_1 <= y) / n for y in sample_2])

        # Розрахунок статистики Смирнова
        D_plus = np.max(np.arange(1, m + 1) / m - F_hat)
        D_minus = np.max(F_hat - (np.arange(1, m + 1) - 1) / m)
        D = max(D_plus, D_minus)

        # Критичне значення
        critical_value = z_gamma * np.sqrt(1 / n + 1 / m)

        # Висновок
        conclusion = "Вибірки однорідні" if D <= critical_value else "Вибірки НЕ однорідні"

        # Вивід у таблиці
        print(f"{n:^20} {λ:^10.2f} {D:^15.5f} {critical_value:^20.5f} {conclusion:^30}")

    print("-" * 95)

 Розмір вибірки (n)      λ             D         Критичне значення              Висновок           
        1000            1.00        0.06700           0.07449              Вибірки однорідні       
        1000            1.20        0.08400           0.07449             Вибірки НЕ однорідні     
-----------------------------------------------------------------------------------------------
       10000            1.00        0.01200           0.02356              Вибірки однорідні       
       10000            1.20        0.07030           0.02356             Вибірки НЕ однорідні     
-----------------------------------------------------------------------------------------------
       100000           1.00        0.00673           0.00745              Вибірки однорідні       
       100000           1.20        0.07187           0.00745             Вибірки НЕ однорідні     
-----------------------------------------------------------------------------------------------


---
# (optional) Помилки 1-го та 2-го роду статистичних тестів

Теоритично, помилка 1-го роду є ~5%, помилка 2-го роду прямує до 0 при $n\to \infty$.

## Колмогоров

In [6]:
import numpy as np
import pandas as pd
from scipy import stats

# Функція для обчислення статистики Колмогорова-Смирнова
def ks_statistic_lib(sample, λ):
    F = lambda x: 1 - np.exp(-λ*x)
    d, _ = stats.kstest(sample, F)
    return d

# Параметри
gamma = 0.05
z_gamma = 1.36  # для gamma=0.05
λ = 1.0

# Визначимо різні значення n
n_values = [100, 500, 1000, 5000, 10000, 50000]
N_dict = {100: 5000, 500: 5000, 1000: 2000, 5000: 1000, 10000: 500, 50000: 500}

results = []

for n in n_values:
    type_1_errors = 0
    type_2_errors = 0
    N = N_dict[n]
    for _ in range(N):
        # Перевірка для помилки першого роду (істинний розподіл)
        sample = np.random.exponential(1/λ, n)
        D_n = ks_statistic_lib(sample, λ)
        value = np.sqrt(n) * D_n
        condition = bool(value >= z_gamma)
        type_1_errors += condition

        # Перевірка для помилки другого роду (відхилення від істинного розподілу)
        sample_alt = np.random.exponential(1.2, n)  # Розподіл зі зміненим параметром λ = 1.2
        D_n_alt = ks_statistic_lib(sample_alt, λ)
        value_alt = np.sqrt(n) * D_n_alt
        condition_alt = bool(value_alt < z_gamma)
        type_2_errors += condition_alt

    type_1_error_rate = type_1_errors / N
    type_2_error_rate = type_2_errors / N

    results.append({
        'n': n,
        'Type I Error': type_1_error_rate,
        'Type II Error': type_2_error_rate
    })

# Побудова таблиці
df = pd.DataFrame(results)
df

,n,Type I Error,Type II Error
0,100,0.0450,0.7308
1,500,0.0452,0.1012
2,1000,0.0505,0.0060
3,5000,0.0540,0.0000
4,10000,0.0520,0.0000
5,50000,0.0560,0.0000


# $\chi^2$

In [7]:
import numpy as np
import pandas as pd
from scipy import stats

# Параметри
gamma = 0.05
λ = 1.0

# Різні значення n для аналізу
n_values = [100, 500, 1000, 5000, 10000, 50000, 100000]
N_dict = {100: 5000, 500: 5000, 1000: 2000, 5000: 1000, 10000: 500, 50000: 500, 100000: 500}
results = []

for n in n_values:
    type_1_errors = 0
    type_2_errors = 0
    N = N_dict[n]
    
    r = int(1 + np.log2(n))  # Кількість інтервалів
    
    for _ in range(N):
        # Критичне значення для критерію хі-квадрат
        z_gamma = stats.chi2.ppf(1 - gamma, df=r-1)

        # Вибірка з правильного розподілу (для перевірки помилки 1 роду)
        sample = np.random.exponential(scale=1/λ, size=n)
        bins = np.quantile(sample, np.linspace(0, 1, r + 1))
        observed, _ = np.histogram(sample, bins=bins)
        expected = n * np.diff(1 - np.exp(-bins))
        chi2_stat = np.sum((observed - expected) ** 2 / expected)
        type_1_errors += (chi2_stat > z_gamma)

        # Вибірка зі зміненого розподілу (для перевірки помилки 2 роду)
        sample_alt = np.random.exponential(scale=1/1.2, size=n)
        bins_alt = np.quantile(sample_alt, np.linspace(0, 1, r + 1))
        observed_alt, _ = np.histogram(sample_alt, bins=bins_alt)
        expected_alt = n * np.diff(1 - np.exp(-bins_alt))
        chi2_stat_alt = np.sum((observed_alt - expected_alt) ** 2 / expected_alt)
        type_2_errors += (chi2_stat_alt <= z_gamma)

    type_1_error_rate = type_1_errors / N
    type_2_error_rate = type_2_errors / N

    results.append({
        'n': n,
        'Type I Error': type_1_error_rate,
        'Type II Error': type_2_error_rate
    })

# Побудова таблиці
df = pd.DataFrame(results)

df

,n,Type I Error,Type II Error
0,100,0.0836,0.8172
1,500,0.0558,0.2880
2,1000,0.0580,0.0215
3,5000,0.0430,0.0000
4,10000,0.0600,0.0000
5,50000,0.0420,0.0000
6,100000,0.0400,0.0000


# Пусті ящики

In [8]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Параметри
gamma = 0.05
λ = 1.2
rho = 2

# Різні значення n для аналізу
n_values = [100, 500, 1000, 5000, 10000, 50000, 100000]
N_dict = {100: 5000, 500: 5000, 1000: 2000, 5000: 1000, 10000: 500, 50000: 500, 100000: 500}

results = []

for n in n_values:
    type_1_errors = 0
    type_2_errors = 0
    N = N_dict[n]
    
    r = n // 2  # Кількість інтервалів
    
    z_gamma = norm.ppf(1 - gamma)
    critical_value = r * np.exp(-rho) + z_gamma * np.sqrt(r * np.exp(-rho) * (1 - (1 + rho) * np.exp(-rho)))
    
    for _ in range(N):
        # Перевірка для помилки 1 роду (істинний розподіл)
        samples = np.random.exponential(scale=1, size=n)
        xi = 1 - np.exp(-samples)  # F(x) для показникового розподілу
        bins = np.linspace(0, 1, r + 1)
        num_observ, _ = np.histogram(xi, bins=bins)
        empty_boxes = np.where(num_observ == 0, 1, 0)
        stat = empty_boxes.sum()
        type_1_errors += (stat > critical_value)

        # Перевірка для помилки 2 роду (відхилення від розподілу)
        samples_alt = np.random.exponential(scale=1/1.2, size=n)
        xi_alt = 1 - np.exp(-samples_alt)
        num_observ_alt, _ = np.histogram(xi_alt, bins=bins)
        empty_boxes_alt = np.where(num_observ_alt == 0, 1, 0)
        stat_alt = empty_boxes_alt.sum()
        type_2_errors += (stat_alt <= critical_value)

    type_1_error_rate = type_1_errors / N
    type_2_error_rate = type_2_errors / N

    results.append({
        'n': n,
        'Type I Error': type_1_error_rate,
        'Type II Error': type_2_error_rate
    })

# Побудова таблиці
df = pd.DataFrame(results)

df

,n,Type I Error,Type II Error
0,100,0.0296,0.9550
1,500,0.0442,0.8892
2,1000,0.0425,0.8490
3,5000,0.0700,0.5020
4,10000,0.0460,0.2620
5,50000,0.0300,0.0000
6,100000,0.0480,0.0000


# Колмогоров-Смирнов

In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Параметри
z_gamma = 1.36  # для gamma=0.05
gamma = 0.05
λ = 1

# Різні значення n для аналізу
n_values = [100, 500, 1000, 5000, 10000, 50000]
N_dict = {100: 5000, 500: 5000, 1000: 2000, 5000: 1000, 10000: 700, 50000: 700}

results = []

for n in n_values:
    type_1_errors = 0
    type_2_errors = 0
    N = N_dict[n]

    m = n // 2

    critical_value = z_gamma * np.sqrt(1 / n + 1 / m)

    for _ in range(N):
        # Генеруємо вибірки для випадку H0
        sample_1 = np.random.exponential(scale=1, size=n)
        sample_2 = np.random.exponential(scale=1, size=m)

        # Сортуємо вибірки
        sample_1 = np.sort(sample_1)
        sample_2 = np.sort(sample_2)

        # Обчислення емпіричної функції розподілу
        F_hat = np.searchsorted(sample_1, sample_2, side='right') / n

        # Розрахунок статистики Смирнова
        D_plus = np.max(np.arange(1, m + 1) / m - F_hat)
        D_minus = np.max(F_hat - (np.arange(1, m + 1) - 1) / m)
        D = max(D_plus, D_minus)

        # Помилка першого роду (істинний розподіл)
        type_1_errors += (D > critical_value)

        # Помилка другого роду (відхилення від розподілу)
        sample_2_alt = np.random.exponential(scale=1/1.2, size=m)  # Розподіл зі зміненим параметром λ = 1.2
        F_hat_alt = np.searchsorted(sample_1, sample_2_alt, side='right') / n
        D_plus_alt = np.max(np.arange(1, m + 1) / m - F_hat_alt)
        D_minus_alt = np.max(F_hat_alt - (np.arange(1, m + 1) - 1) / m)
        D_alt = max(D_plus_alt, D_minus_alt)
        type_2_errors += (D_alt <= critical_value)

    type_1_error_rate = type_1_errors / N
    type_2_error_rate = type_2_errors / N

    results.append({
        'n': n,
        'Type I Error': type_1_error_rate,
        'Type II Error': type_2_error_rate
    })

# Побудова таблиці
df = pd.DataFrame(results)

df

,n,Type I Error,Type II Error
0,100,0.037200,0.0
1,500,0.041000,0.0
2,1000,0.047500,0.0
3,5000,0.054000,0.0
4,10000,0.038571,0.0
5,50000,0.055714,0.0
